## Load Packages

In [1]:
using StatsBase

include("printmat.jl")

printlnPs (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

# Loading Daily S&P 500 Data

In [3]:
xx = readdlm("Data/SP500RfPs.csv",',',header=true)
x  = xx[1]
SP = convert(Array{Float64},x[:,2])      #convert to numerical array, S&P 500 level
R  = (SP[2:end]./SP[1:end-1] - 1) * 100  #returns, % 
T  = length(R)  
println("Number of days in the sample: ",T)

dN = Array{Date}(T)                      #convert to Date
for t = 1:T
    dN[t] = Date(x[t+1,1],"d/m/y")
end

Number of days in the sample: 9352


# Autocorrelations

$\rho_s = \text{Corr}(R_t,R_{t-s})$

In large samples, $\sqrt{T}\hat{\rho}_{s}\sim N(0,1)$

In [4]:
plags = 1:5
ρ = autocor(R,plags)
println("\nlag, autocorrr, t-stat(autocorr) of excess returns")
printmat([plags ρ sqrt(T)*ρ])


lag, autocorrr, t-stat(autocorr) of excess returns
     1.000    -0.026    -2.480
     2.000    -0.044    -4.208
     3.000    -0.007    -0.684
     4.000    -0.022    -2.124
     5.000    -0.018    -1.744



# Autoregressions

An AR(1) is 

$R_{t}=c+a_{1}R_{t-1}+\varepsilon_{t}$

and can be estimated by OLS

An asymmteric AR(1) 

$R_{t} =\alpha+\beta Q_{t-1}R_{t-1}+\gamma(1-Q_{t-1})R_{t-1}+\varepsilon
_{t}$,

where $Q_{t-1}=1 \ \text{ if } \ R_{t-1}<0$ and zero otherwise

In [5]:
function OlsFn(y,x)                      #OLS estimation 
    T = size(y,1)                        #alternative: use GLM.jl
    b     = x\y
    yhat  = x*b
    res   = y - yhat
    Covres = cov(res)
    Covb   = kron(Covres,inv(x'x))       #works also if y is Txn
    R2a    = 1 - var(res,1)./var(y,1)
    return b,res,yhat,Covb,R2            #the outputs
end

OlsFn (generic function with 1 method)

In [6]:
y  = R[2:end]
Tb = size(y,1)
x  = [ones(Tb) R[1:end-1]]
(b,res,yhat,Covb,) = OlsFn(y,x) 
Stdb  = sqrt(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1): intercept and slope with t-stats")
printmat([b tstat])

Results from AR(1): intercept and slope with t-stats

In [7]:
DummyNegative = R[1:end-1] .< 0

x = [ones(Tb) DummyNegative.*R[1:end-1] (1-DummyNegative).*R[1:end-1]]

(b,res,yhat,Covb,) = OlsFn(y,x) 
Stdb  = sqrt.(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1) with dummies: [intercept, slope neg, slope pos] with t-stats")
printmat([b tstat])

Results from AR(1) with dummies: [intercept, slope neg, slope pos] with t-stats
     0.001     0.058
    -0.075    -4.395
     0.027     1.518



## Recursive Estimation and Out-of-Sample R2

Do recursive estimation (longer and longer sample) an predict one period ahead (outside of the sample). Define an "out-of-sample" $R^2$ as 

$R_{OS}^2 = 1- \frac{\text{MSE(forecasting model)}}{\text{MSE(benchmark forecast)}}$

In [8]:
y = R[2:end]
Tb = size(y,1)
x = [ones(Tb,1) R[1:end-1]]

bT     = fill(NaN,size(x))
rHat   = fill(NaN,Tb)
rTilde = fill(NaN,Tb)
for t = 100:Tb
    (b,res,yhat,Covb,) = OlsFn(y[1:t-1],x[1:t-1,:]) 
    bT[t,:] = b'
    rHat[t] = (x[t:t,:]*b)[1]           #out-of-sample forecast of period t
    rTilde[t] = mean(y[1:t-1])          #benchmark forecast: historical average
end    

MSE_Model = mean((y[100:end]-rHat[100:end]).^2)    #MSE for out-of-sample period
MSE_Bench = mean((y[100:end]-rTilde[100:end]).^2)

printlnPs("MSE of AR(1) model and the benchmark: ",[MSE_Model MSE_Bench])
printlnPs("out-of-sample R2: ",1-MSE_Model/MSE_Bench)

MSE of AR(1) model and the benchmark:      1.247     1.244
out-of-sample R2:     -0.002


In [9]:
YearFrac  = Dates.year(dN) + (Dates.month(dN)-1)/12 + (Dates.day(dN)-1)/(31*12)

plot(YearFrac[2:end],bT[:,1],color=:blue,legend=false)
title!("Estimated intercept")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 Estimated intercept 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 45.3453,287.773 45.5067,273.323 45.5471,284.51 45.5874,336.057 45.6278,334.787 45.7489,330.587 45.7892,323.817 45.8296,258.593 45.87,233.952 45.9103,215.625 
 46.0314,233.349 46.0718,212.342 46.1122,172.598 46.1525,200.477 46.1929,205.731 46.314,212.302 46.3543,238.572 46.3947,236.879 46.4351,236.102 46.4754,216.895 
 46.5965,196.714 46.6369,223.051 46.6772,242.761 46.7176,214.293 46.758,196.454 46.9194,195.497 46.9598,236.119 47.0405,229.203 47.0809,216.767 47.202,171.782 
 47.2423,148.035 47.2827,162.446 47.3231,186.64 47.3634,226.86 47.4845,240.023 47.5249,222.063 47.5652,259.244 47.6056,262.188 47.646,265.806 47.7671,258.243 
 47.8074,268.83 47.8478,253.945 47.8881,216.013 47.9285,220.666 48.0496,222.038 48.09,221.471 48.1303,198.968 48.1707,190.837 48.2111,196.01 48.3321,199.308 
 48.3725,191.212 48.4129,148.048 48.4532,144.807 48.4936,163.552 48.6147,129.977 48.655,105.255 48.6954,107.532 48.7358,86.273 48.7761,95.0282 48.8972,89.4796 
 48.9376,75.8519 48.978,77.2286 49.0183,77.0722 49.0587,90.4635 49.1798,92.1499 49.2201,77.1745 49.2605,84.0135 49.3009,85.5882 49.3412,87.5264 49.5027,80.4059 
 49.543,136.11 49.5834,170.963 49.6238,152.206 49.7449,132.626 49.7852,123.614 49.8256,144.555 49.8659,133.548 49.9063,135.26 50.0274,110.836 50.0678,113.038 
 50.1081,138.334 50.1485,127.809 50.1889,70.501 50.3099,76.9117 50.3503,102.23 50.3907,98.886 50.431,93.2481 50.4714,89.2734 50.6329,114.404 50.6732,136.369 
 50.7136,104.532 50.7539,108.381 50.7943,94.7152 50.9154,71.8153 50.9558,104.619 50.9961,204.343 51.0365,236.706 51.0768,238.211 51.1979,253.004 51.2383,282.332 
 51.2787,281.912 51.319,276.343 51.3594,272.252 51.4805,325.349 51.5208,341.423 51.5612,348.912 51.6016,342.618 51.6419,355.209 51.763,337.942 51.8034,337.476 
 51.8438,288.179 51.8841,315.829 51.9245,292.59 52.0456,297.43 52.0859,314.96 52.1263,328.618 52.1667,360.648 52.207,343.533 52.3281,315.848 52.3685,274.902 
 52.4088,295.226 52.4492,282.134 52.4896,267.317 52.6107,278.329 52.651,266.95 52.6914,281.405 52.7721,275.007 52.8932,258.475 52.9336,215.868 52.9739,231.118 
 53.0143,221.293 53.0547,222.555 53.2161,237.807 53.2565,243.844 53.2968,221.565 53.3372,216.158 53.3776,202.901 53.4987,215.702 53.539,211.483 53.5794,216.568 
 53.6197,216.108 53.6601,213.83 53.7812,189.175 53.8216,186.174 53.8619,208.652 53.9023,208.152 53.9426,207.39 54.0637,222.041 54.1445,219.021 54.1848,217.553 
 54.2252,215.057 54.3463,218.755 54.427,217.085 54.4674,261.476 54.5077,267.014 54.6288,238.267 54.6692,237.278 54.7096,197.112 54.7499,201.597 54.7903,186.525 
 54.9114,189.076 54.9517,181.227 54.9921,169.567 55.0325,174.012 55.0728,181.25 55.1939,173.931 55.2343,157.378 55.2746,170.478 55.315,132.126 55.3554,133.204 
 55.4765,136.396 55.5168,114.823 55.5572,130.139 55.5975,107.594 55.6379,126.48 55.759,106.483 55.7994,120.698 55.8397,114.845 55.8801,98.7007 55.9205,93.0534 
 56.0415,68.7353 56.0819,83.1751 56.1223,68.5583 56.1626,62.7833 56.203,91.0651 56.3644,114.986 56.4048,128.836 56.4452,94.325 56.4855,113.188 56.6066,116.743 
 56.647,146.25 56.6874,131.552 56.7277,158.027 56.7681,157.001 56.9699,136.011 57.0103,154.99 57.0506,149.256 57.091,176.745 57.1314,221.956 57.2524,252.68 
 57.2928,256.922 57.3332,234.511 57.3735,251.267 57.4139,272.344 57.535,273.769 57.5754,328.206 57.6157,286.09 57.6561,285.479 57.6964,306.171 57.8175,319.35 
 57.8579,372.012 57.8983,368.557 57.9386,377.523 57.979,384.952 58.1001,340.508 58.1404,321.555 58.1808,322.721 58.2212,314.739 58.3826,324.564 58.423,357.256 
 58.4633,335.902 58.5037,306.495 58.5441,294.984 58.6652,301.518 58.7055,316.903 58.7459,318.987 58.7863,336.76 58.8266,343.237 58.9477,350.836 58.9881,362.558 
 5

In [10]:
plot(YearFrac[2:end],bT[:,2],color=:red,legend=false)
title!("Estimated AR(1) slope")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 Estimated AR(1) slope 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 45.3453,231.605 45.5067,231.547 45.5471,233.694 45.5874,225.954 45.6278,226.805 45.7489,226.978 45.7892,226.645 45.8296,220.443 45.87,194.182 45.9103,185.025 
 46.0314,191.046 46.0718,196.169 46.1122,184.794 46.1525,205.245 46.1929,203.148 46.314,202.386 46.3543,199.336 46.3947,200.015 46.4351,200.029 46.4754,199.949 
 46.5965,193.281 46.6369,203.951 46.6772,195.621 46.7176,205.411 46.758,197.137 46.9194,196.762 46.9598,198.56 47.0405,203.133 47.0809,202.603 47.202,192.011 
 47.2423,167.625 47.2827,177.286 47.3231,171.581 47.3634,154.459 47.4845,145.7 47.5249,151.077 47.5652,162.109 47.6056,160.19 47.646,159.704 47.7671,160.384 
 47.8074,161.809 47.8478,164.829 47.8881,154.348 47.9285,158.72 48.0496,158.691 48.09,158.712 48.1303,158.67 48.1707,154.488 48.2111,155.77 48.3321,155.458 
 48.3725,156.164 48.4129,148.937 48.4532,144.903 48.4936,148.869 48.6147,161.949 48.655,143.457 48.6954,145.403 48.7358,144.999 48.7761,149.857 48.8972,150.805 
 48.9376,149.423 48.978,149.933 49.0183,149.933 49.0587,149.911 49.1798,149.389 49.2201,150.615 49.2605,153.273 49.3009,153.038 49.3412,152.92 49.5027,153.356 
 49.543,163.386 49.5834,124.701 49.6238,138.09 49.7449,131.956 49.7852,126.191 49.8256,133.678 49.8659,139.063 49.9063,139.39 50.0274,140.096 50.0678,141.728 
 50.1081,141.849 50.1485,148.116 50.1889,138.606 50.3099,154.886 50.3503,152.74 50.3907,154.651 50.431,154.677 50.4714,154.08 50.6329,157.268 50.6732,144.199 
 50.7136,160.998 50.7539,164.135 50.7943,164.846 50.9154,155.813 50.9558,184.703 50.9961,102.954 51.0365,53.922 51.0768,52.7618 51.1979,50.0111 51.2383,39.1453 
 51.2787,39.4353 51.319,40.1738 51.3594,39.7271 51.4805,46.2298 51.5208,27.6009 51.5612,23.3815 51.6016,25.1925 51.6419,26.3548 51.763,31.4718 51.8034,31.298 
 51.8438,27.4149 51.8841,66.4702 51.9245,78.6088 52.0456,80.9068 52.0859,79.9182 52.1263,74.004 52.1667,61.3624 52.207,75.1735 52.3281,67.7835 52.3685,35.1942 
 52.4088,63.0073 52.4492,67.5066 52.4896,63.7172 52.6107,68.5668 52.651,71.0346 52.6914,74.6106 52.7721,76.7159 52.8932,75.0621 52.9336,55.0267 52.9739,77.4864 
 53.0143,79.9844 53.0547,80.2378 53.2161,80.1646 53.2565,77.8985 53.2968,82.5526 53.3372,79.4634 53.3776,76.2267 53.4987,81.6812 53.539,82.8951 53.5794,83.1861 
 53.6197,83.2457 53.6601,83.2719 53.7812,81.8346 53.8216,79.493 53.8619,84.1806 53.9023,84.459 53.9426,84.5245 54.0637,84.5912 54.1445,85.7676 54.1848,85.7496 
 54.2252,85.6459 54.3463,85.931 54.427,86.0945 54.4674,87.4244 54.5077,81.3789 54.6288,90.6911 54.6692,89.8866 54.7096,84.0673 54.7499,90.5662 54.7903,90.1766 
 54.9114,91.3673 54.9517,91.4822 54.9921,88.8 55.0325,90.6338 55.0728,90.0455 55.1939,91.6244 55.2343,88.7054 55.2746,96.4498 55.315,109.216 55.3554,110.58 
 55.4765,110.907 55.5168,112.713 55.5572,123.657 55.5975,132.839 55.6379,144.784 55.759,155.058 55.7994,162.389 55.8397,164.677 55.8801,162.797 55.9205,159.523 
 56.0415,153.1 56.0819,167.7 56.1223,173.747 56.1626,171.521 56.203,178.373 56.3644,160.674 56.4048,152.339 56.4452,166.403 56.4855,187.763 56.6066,185.812 
 56.647,181.367 56.6874,192.379 56.7277,200.604 56.7681,201.337 56.9699,202.282 57.0103,215.731 57.0506,218.839 57.091,221.9 57.1314,188.607 57.2524,156.49 
 57.2928,153.131 57.3332,157.794 57.3735,168.955 57.4139,159.642 57.535,158.796 57.5754,152.787 57.6157,210.702 57.6561,210.117 57.6964,211.927 57.8175,204.65 
 57.8579,184.169 57.8983,188.791 57.9386,188.357 57.979,186.516 58.1001,196.528 58.1404,173.155 58.1808,173.932 58.2212,173.682 58.3826,175.848 58.423,167.83 
 58.4633,185.959 58.5037,172.276 58.5441,161.915 58.6652,164.71 58.7055,162.623 58.7459,161.768 58.7863,159.966 58.8266,156.898 58.9477,155.204 58.9881,